In [1]:
#CoLab Only
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
#CoLab Only
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"
import findspark
findspark.init()

В этой практике мы применим наши знания по PySpark и постараемся изучить что-то новое в процессе выполнения.
В занятии используется датасет собранный на основе данных Chicago Taxi Rides 2016  
Схема данных:  
|-- taxi_id = идентификатор таксиста  
|-- trip_start_timestamp = время начала поездки  
|-- trip_end_timestamp = время окончания поездки  
|-- trip_seconds = время длительности поездки в секундах  
|-- trip_miles = мили проиденные во время поездки  
|-- fare = транспортные расходы  
|-- tips = назначенные чаевые  
|-- trip_total = общая стоимость поездки (Итоговая с учетом чаевых и расходов)  
|-- payment_type = тип оплаты  

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [134]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, hour

In [ ]:
spark = SparkSession.builder.appName('PySparkTasks').getOrCreate()

In [ ]:
spark.conf.set("spark.sql.session.timeZone", "GMT+3")#конфигурируем Спарк, устанавливаем таймзону

Загружаем данные

In [15]:
df = spark.read.parquet('/content/drive/My Drive/Data for Collab/PySpark/taxi_data.parquet')

№1 Посчитайте количество загруженных строк.

In [17]:
df.count()

2540712

In [19]:
df.show(5)

+-------+--------------------+-------------------+------------+----------+-----+----+----------+------------+
|taxi_id|trip_start_timestamp| trip_end_timestamp|trip_seconds|trip_miles| fare|tips|trip_total|payment_type|
+-------+--------------------+-------------------+------------+----------+-----+----+----------+------------+
|   5240| 2016-12-15 20:45:00|2016-12-15 21:00:00|         900|       2.5|10.75|2.45|      14.7| Credit Card|
|   1215| 2016-12-12 04:15:00|2016-12-12 04:15:00|         240|       0.4|  5.0| 3.0|       9.5| Credit Card|
|   3673| 2016-12-16 13:30:00|2016-12-16 14:00:00|        2400|      10.7| 31.0| 0.0|      31.0|        Cash|
|   5400| 2016-12-16 05:45:00|2016-12-16 06:00:00|         300|       0.0| 5.25| 2.0|      7.25| Credit Card|
|   1257| 2016-12-03 15:45:00|2016-12-03 15:45:00|         360|       0.3|  5.0| 0.0|       5.0|        Cash|
+-------+--------------------+-------------------+------------+----------+-----+----+----------+------------+
only showi

№2 Чему равна корреляция и ковариация между длиной маршрута и ценой за поездку? Ответ округлите до 5 знаков после запятой.


In [23]:
corr_=round(df.corr('trip_miles','trip_total'),5)
corr_

0.44816

In [25]:
cov_=round(df.cov('trip_miles','trip_total'),5)
cov_

71.96914

№3 Найдите количество, среднее, cреднеквадратическое отклонение, минимум и максимум для длины маршрута и цены за поездку? Ответ округлите до 1 знака после запятой.

In [29]:
df[['trip_miles','trip_total']].describe().show()

+-------+------------------+------------------+
|summary|        trip_miles|        trip_total|
+-------+------------------+------------------+
|  count|           2540677|           2540672|
|   mean|3.0005873828090266|15.913560215564042|
| stddev|  5.25716922943536|30.546699217618237|
|    min|               0.0|               0.0|
|    max|             900.0|           9276.69|
+-------+------------------+------------------+



№4 Найдите самый НЕпопулярный вид оплаты.

In [39]:
df.groupby('payment_type')\
  .agg({'trip_miles':'count'})\
  .orderBy('count(trip_miles)')\
  .show()

+------------+-----------------+
|payment_type|count(trip_miles)|
+------------+-----------------+
|    Way2ride|                3|
|       Pcard|              878|
|      Prcard|              968|
|     Dispute|             1842|
|     Unknown|             5179|
|   No Charge|            12843|
| Credit Card|          1108842|
|        Cash|          1410122|
+------------+-----------------+



№5 Найдите идентификатор таксиста выполнившего наибольшее число заказов.

In [47]:
df.groupby('taxi_id')\
  .agg({'trip_miles':'count'})\
  .orderBy('count(trip_miles)', ascending=False)\
  .show()

+-------+-----------------+
|taxi_id|count(trip_miles)|
+-------+-----------------+
|    316|             2225|
|   6591|             2083|
|   5071|             2080|
|   8740|             2067|
|   6008|             2033|
|   8629|             2024|
|   1462|             2007|
|    375|             1986|
|   8751|             1938|
|   5357|             1930|
|   8264|             1909|
|   1168|             1809|
|   1946|             1803|
|    336|             1799|
|   1521|             1799|
|   3253|             1764|
|   8561|             1760|
|   8344|             1743|
|   8496|             1742|
|   6482|             1740|
+-------+-----------------+
only showing top 20 rows



№6 Чему равна средняя цена среди поездок, оплаченных наличными? Ответ округлите до 5 знака.


In [70]:
round(df.where(df.payment_type =='Cash')\
  .agg({'trip_total':'avg'}).collect()[0][0],5)

12.03526

№7 Сколько таксистов проехало больше 1000 миль за все время выполнения заказов?

In [85]:
gdf = df.groupby('taxi_id')\
  .agg({'trip_miles':'sum'})\
    .withColumnRenamed("sum(trip_miles)", "miles")
#gdf.orderBy(gdf.miles>1000).show()
gdf.where(gdf.miles>1000).count()

2860

№8 Сколько миль проехал пассажир в самой долгой поездке? (Ответ округлите до целого)

In [92]:
max_time = df.agg({'trip_seconds':'max'}).collect()[0][0]
df.where(df.trip_seconds==max_time).show()

+-------+--------------------+-------------------+------------+----------+----+----+----------+------------+
|taxi_id|trip_start_timestamp| trip_end_timestamp|trip_seconds|trip_miles|fare|tips|trip_total|payment_type|
+-------+--------------------+-------------------+------------+----------+----+----+----------+------------+
|   4161| 2016-11-14 13:00:00|2016-11-15 13:00:00|       86399|       0.0|3.25| 0.0|      3.25|        Cash|
|   1954| 2016-11-02 21:15:00|2016-11-03 21:15:00|       86399|       0.0|3.25| 0.0|      3.25|        Cash|
|   5667| 2016-11-04 18:30:00|2016-11-05 18:30:00|       86399|       0.0|3.25| 0.0|      4.75|        Cash|
+-------+--------------------+-------------------+------------+----------+----+----+----------+------------+



№9 Каков средний заработок всех таксистов? Ответ округлите до 5-ого знака.
Отсеките неизвестные машины (не определенный taxi_id).

In [98]:
round(df.dropna().agg({'trip_total':'avg'}).collect()[0][0],5)

15.91157

№10 Сколько поездок начиналось в самый загруженный час?
Используйте функцию hour

In [139]:
dfh = df.withColumn('hour', hour(col('trip_start_timestamp')))

In [143]:
dfh.groupby('hour').agg({'fare':'count'}).orderBy('count(fare)', ascending=False).show(1)

+----+-----------+
|hour|count(fare)|
+----+-----------+
|  15|     181124|
+----+-----------+
only showing top 1 row



№12 Найдите топ три даты, в которые было суммарно больше всего чаевых? (Чаевые выдаются после совершения поездки)
Ожидаемый формат дат YYYY-MM-DD

In [99]:
from pyspark.sql.types import DateType

In [100]:
df.dtypes

[('taxi_id', 'int'),
 ('trip_start_timestamp', 'timestamp'),
 ('trip_end_timestamp', 'timestamp'),
 ('trip_seconds', 'int'),
 ('trip_miles', 'double'),
 ('fare', 'double'),
 ('tips', 'double'),
 ('trip_total', 'double'),
 ('payment_type', 'string')]

In [102]:
df.show(2)

+-------+--------------------+-------------------+------------+----------+-----+----+----------+------------+
|taxi_id|trip_start_timestamp| trip_end_timestamp|trip_seconds|trip_miles| fare|tips|trip_total|payment_type|
+-------+--------------------+-------------------+------------+----------+-----+----+----------+------------+
|   5240| 2016-12-15 20:45:00|2016-12-15 21:00:00|         900|       2.5|10.75|2.45|      14.7| Credit Card|
|   1215| 2016-12-12 04:15:00|2016-12-12 04:15:00|         240|       0.4|  5.0| 3.0|       9.5| Credit Card|
+-------+--------------------+-------------------+------------+----------+-----+----+----------+------------+
only showing top 2 rows



In [113]:
df.withColumn('trip_end_timestamp',df.trip_end_timestamp.cast(DateType()))\
  .groupby('trip_end_timestamp')\
  .agg({'tips':'sum'})\
  .orderBy('sum(tips)', ascending=False)\
  .show(3)

+------------------+------------------+
|trip_end_timestamp|         sum(tips)|
+------------------+------------------+
|        2016-11-09|106671.48999999987|
|        2016-11-03|101606.06000000023|
|        2016-11-16|100606.96000000041|
+------------------+------------------+
only showing top 3 rows



№13 Сколько было заказов в дату с наибольшим спросом?

In [117]:
cdf = df.withColumn('trip_start_timestamp',df.trip_start_timestamp.cast(DateType()))
cdf.groupby('trip_start_timestamp').agg({'fare':'count'}).orderBy('count(fare)', ascending=False).show(1)

+--------------------+-----------+
|trip_start_timestamp|count(fare)|
+--------------------+-----------+
|          2016-12-16|      62184|
+--------------------+-----------+
only showing top 1 row



№14 Какая марка машины самая распрастранненая среди таксистов?

In [118]:
df_cars = spark.read.parquet('/content/drive/My Drive/Data for Collab/PySpark/taxi_cars_data.parquet')

In [120]:
df_cars.show(2)

+-------+-------------------+
|taxi_id|          car_model|
+-------+-------------------+
|   1159|       Toyota Prius|
|   7273|Ford Crown Victoria|
+-------+-------------------+
only showing top 2 rows



In [127]:
df_cars_new = df_cars.withColumn('mark_auto', split(col('car_model'),' ')[0])
df_cars_new.groupby('mark_auto').agg({'taxi_id':'count'}).orderBy('count(taxi_id)', ascending=False).show()

+----------+--------------+
| mark_auto|count(taxi_id)|
+----------+--------------+
|      Ford|          1483|
|   Hyundai|           792|
|    Toyota|           691|
| Chevrolet|           473|
|       Kia|           265|
|      Audi|           250|
|   Lincoln|           247|
|     Honda|           246|
|Volkswagen|           244|
|    Nissan|           225|
+----------+--------------+



№15 Сколько раз и какая модель машин чаще всего встречается в поездках?

In [128]:
jdf = df.join(df_cars, on='taxi_id')

In [132]:
jdf.groupby('car_model').agg({'taxi_id':'count'}).orderBy('count(taxi_id)', ascending=False).show(2)

+-------------------+--------------+
|          car_model|count(taxi_id)|
+-------------------+--------------+
|Ford Crown Victoria|        388682|
|     Hyundai Accent|        150764|
+-------------------+--------------+
only showing top 2 rows



Почувствуй силу сжатия! сохрани DataFrame в csv и сравни размеры файлов.

In [133]:
df.coalesce(1).write.csv('taxi_L_csv')